<a href="https://colab.research.google.com/github/haiyuhuang177/Hangman-OOP-with-N-gram-/blob/main/OOP_Hangman.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Objected-Oriented Hangman with N-gram model

In [153]:
from collections import Counter, defaultdict
from itertools import product
import string
import random
import re

In [154]:
def hangman(secret, guesser, max_mistakes=6, recalibrate=False, verbose=True):
    """
        This function plays the hangman game with the provided guesser and returns the number of incorrect guesses.
        secret: a string of alphabetic characters, i.e., the answer to the game
        guesser: an object of the class Guesser which guesses the next character at each stage in the game
        max_mistakes: limit on length of game, in terms of number of allowed mistakes
        recalibrate: whether to dynamically update the guesser's dictionary
        verbose: silent or verbose diagnostic prints
    """
    secret = secret.lower()
    masked_word = ['_'] * len(secret)
    guessed = set()
    if recalibrate:
        guesser.adapt(guesser.original_corpus) # start from the original dictionary if recalibrate
    if verbose:
        print("Starting hangman game. Target is", ' '.join(masked_word), 'of length', len(secret))

    mistakes = 0
    while mistakes < max_mistakes:
        if verbose:
            print("You have", (max_mistakes - mistakes), "attempts remaining.")

        guess = guesser.guess(masked_word, guessed) # make a guess with guesser
        if verbose:
            print('Your guess is', guess)
        if guess in guessed:
            if verbose:
                print('Already guessed this before.')
            mistakes += 1
        else:
            guessed.add(guess)
            if guess in secret and len(guess) == 1: # correct guess
                for i, char in enumerate(secret):
                    if char == guess:
                        masked_word[i] = char # reveal correctly guessed letters
                if verbose:
                    print('Good guess:', ' '.join(masked_word))
            else: # incorrect guess
                if len(guess) != 1:
                    print('Please guess with only 1 character.')
                if verbose and mistakes < max_mistakes:
                    print('Sorry, try again.')

                mistakes += 1
            if recalibrate:
                guesser.recalibrate(masked_word, guessed)

        if '_' not in masked_word:
            if verbose:
                print('Congratulations! You won.')
            return mistakes

    if verbose:
        print('Out of guesses. The secret word was', secret)
    return mistakes

We define a Guesser class with HumanGuesser and RandomGuesser as subclass. The HumanGuesser allows interactive play and RandomGuesser guesses randomly from the set of letters that are not in the guessed set.

In [155]:
# @title Guesser class
class Guesser:
    def __init__(self):
        pass

    def guess(self, masked_word, guessed):
        pass

class HumanGuesser(Guesser):
    def guess(self, masked_word, guessed):
        print('\nEnter your guess:')
        return input().lower().strip()

class RandomGuesser(Guesser):
    def __init__(self):
        self.name = "random"

    def guess(self, masked_word, guessed):
        available = list(set(string.ascii_lowercase) - guessed)
        return random.choice(available)

If you want to play hangman interactively, instantiate a human guesser and set `interactive` to `True`.

In [138]:
interactive = False

In [ ]:
human = HumanGuesser()
if interactive:
    hangman('whatever', human, 6, True)

## Pre-processing corpus
One can pick any set of English words for training. Here we use NLTK's Brown corpus for training and for evaluating the quality of the algorithm.

In [3]:
import numpy as np
import nltk
from nltk.corpus import brown

np.random.seed(1)

training_set = []
test_set = []

nltk.download('brown')
brown_words = brown.words()

# lowercase the corpus and remove the word which contain non-alphabetic characters
processed_words = []
for word in brown_words:
    if word.isalpha():
        processed_words.append(word.lower())

# unique words in brown corpus
unique_words = list(set(processed_words)) # 40234
# change to array in order to conduct np shuffle, then convert to list
unique_words = np.array(unique_words)
np.random.shuffle(unique_words)
unique_words = unique_words.tolist()

test_set, training_set = unique_words[:1000], unique_words[1000:]

print("Number of words in training set is", len(training_set))
print("Number of words in test set is", len(test_set))

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.


Number of words in training set is 39234
Number of words in test set is 1000


**Play hangman using random words from test set**

In [ ]:
if interactive:
    hangman(np.random.choice(test_set), human, 6, True)

### Testing Random Guesser

Baseline is a trivial **random method**. The random guesser randomly chooses a character from `a ... z` after excluding the characters that have already been guessed in the current game.

`test_guesser` method that takes a guesser and measures the average number of guesses made over all the words in the `test_set` and return the proportion of successful games with less than equal to 6 attempts.

In [156]:
def test_guesser(guesser, test_set, recalibrate = False, verbose = False):
    total = 0
    num_success = 0
    recalibrate_str = " with recalibration" if recalibrate else ""
    print(f"\nTesting the {guesser.name} guesser using every word in test set" + recalibrate_str)
    for word in test_set:
        num_guesses = hangman(word, guesser, 26, recalibrate = recalibrate, verbose = verbose)
        total += num_guesses
        num_success += int(num_guesses <= 6)
    print("Average number of guesses used: ", total / float(len(test_set)))
    print("Success rate with 6 tries: ", num_success / float(len(test_set)))
    return num_success / float(len(test_set))

In [141]:
random_guesser = RandomGuesser()
random_word = np.random.choice(test_set)
print("Guessing word =", random_word)
print("Number of mistakes made by the random guesser =", hangman(random_word, random_guesser, 26, False, False))

test_guesser(random_guesser, test_set)

Guessing word = interclass
Number of mistakes made by the random guesser = 15

Testing the random guesser using every word in test set
Average number of guesses used:  16.493
Success rate with 6 tries:  0.009


0.009

## Unigram Model

Unigram finds the frequencies of characters over all training words. It returns the character with the highest frequency after excluding guessed characters.

In [157]:
class UnigramGuesser(Guesser):
    def __init__(self, by_length = False):
        self.name = "unigram"
        self.map = {}               # mapping letter to frequencies
        self.map_len = {}           # mapping length to letter frequencies
        self.char_prob = {}         # likelihood of each letter
        self.corpus = []            # may change if recalibrate = True
        self.original_corpus = []   # reset to the original corpus if recalibrate is turned on
        self.by_length = by_length  # whether or not to condition on length

    def upload(self, corpus):
        ''' store the original corpus '''
        self.original_corpus = corpus
        self.adapt(corpus)

    def adapt(self, corpus):
        ''' training the model on corpus '''
        self.corpus = corpus
        self.map = Counter()
        self.map_len = defaultdict(Counter)
        for word in corpus:
            length = len(word)
            for char in word:
                self.map[char] += 1
                self.map_len[length][char] += 1

    def recalibrate(self, masked_word, guessed):
        ''' update the dictionary after each guess '''
        incorrect = guessed - set(masked_word)
        self.corpus = [word for word in self.corpus if not set(word).intersection(incorrect)] #and bool(re.fullmatch("".join(masked_word).replace('_', '.'), word))]
        self.adapt(self.corpus)

    def update_prob(self, masked_word, guessed):
        ''' update the likelihood of each letter '''
        self.char_prob = {x : 0 for x in string.ascii_lowercase}
        if not self.by_length or len(masked_word) not in self.map_len.keys():
            curr_dict = self.map
        else:
            curr_dict = self.map_len[len(masked_word)]
        total_count = 0
        for char in self.char_prob:
            if char not in guessed:
                self.char_prob[char] = curr_dict[char]
                total_count += curr_dict[char]
        if total_count > 0:
            self.char_prob = {x : self.char_prob[x] / total_count for x in self.char_prob}

    def guess(self, masked_word, guessed):
        ''' return the most probable letter as the guess '''
        self.update_prob(masked_word, guessed)
        # in case char_prob is identically zero because the guesser has guessed all letters
        # (of a specific length) in the corpus
        if list(self.char_prob.values()) == [0] * len(self.char_prob):
            filtered = {x : self.char_prob[x] for x in self.char_prob if x not in guessed}
            return max(filtered, key = filtered.get)
        return max(self.char_prob, key=self.char_prob.get)

In [98]:
unigram_guesser = UnigramGuesser()
unigram_guesser.upload(training_set)
test_guesser(unigram_guesser, test_set, recalibrate = False)


Testing the unigram guesser using every word in test set
Average number of guesses used:  10.226
Success rate with 6 tries:  0.187


0.187

In [158]:
hangman('whatever', unigram_guesser, 6, recalibrate = False, verbose = True)
hangman('whatever', unigram_guesser, 6, recalibrate = True, verbose = True)
hangman('unquestionably', unigram_guesser, 6, recalibrate = True, verbose = True)

Starting hangman game. Target is _ _ _ _ _ _ _ _ of length 8
You have 6 attempts remaining.
Your guess is e
Good guess: _ _ _ _ e _ e _
You have 6 attempts remaining.
Your guess is o
Sorry, try again.
You have 5 attempts remaining.
Your guess is d
Sorry, try again.
You have 4 attempts remaining.
Your guess is t
Good guess: _ _ _ t e _ e _
You have 4 attempts remaining.
Your guess is u
Sorry, try again.
You have 3 attempts remaining.
Your guess is m
Sorry, try again.
You have 2 attempts remaining.
Your guess is b
Sorry, try again.
You have 1 attempts remaining.
Your guess is p
Sorry, try again.
Out of guesses. The secret word was whatever
Starting hangman game. Target is _ _ _ _ _ _ _ _ of length 8
You have 6 attempts remaining.
Your guess is e
Good guess: _ _ _ _ e _ e _
You have 6 attempts remaining.
Your guess is i
Sorry, try again.
You have 5 attempts remaining.
Your guess is a
Good guess: _ _ a _ e _ e _
You have 5 attempts remaining.
Your guess is r
Good guess: _ _ a _ e _ e r
You

6

The accuracy increases from around 19% to 21% with recalibration. However, running the test takes significantly longer with recalibration. We may also condition on the length of the secret word. Different lengths tend to have different character distributions. When encounter an unseen word length, the method reverts back to  `unigram_guesser`. One see that the improvement, if any, is rather minor.

In [149]:
unigram_len_guesser = UnigramGuesser(by_length=True)
unigram_len_guesser.upload(training_set)
test_guesser(unigram_len_guesser, test_set, recalibrate = False)


Testing the unigram guesser using every word in test set
Average number of guesses used:  10.172
Success rate with 6 tries:  0.199


0.199

In [159]:
hangman('whatever', unigram_len_guesser, 6, recalibrate = True, verbose = True)
hangman('unquestionably', unigram_len_guesser, 6, recalibrate = True, verbose = True)

Starting hangman game. Target is _ _ _ _ _ _ _ _ of length 8
You have 6 attempts remaining.
Your guess is e
Good guess: _ _ _ _ e _ e _
You have 6 attempts remaining.
Your guess is i
Sorry, try again.
You have 5 attempts remaining.
Your guess is a
Good guess: _ _ a _ e _ e _
You have 5 attempts remaining.
Your guess is r
Good guess: _ _ a _ e _ e r
You have 5 attempts remaining.
Your guess is s
Sorry, try again.
You have 4 attempts remaining.
Your guess is o
Sorry, try again.
You have 3 attempts remaining.
Your guess is d
Sorry, try again.
You have 2 attempts remaining.
Your guess is l
Sorry, try again.
You have 1 attempts remaining.
Your guess is t
Good guess: _ _ a t e _ e r
You have 1 attempts remaining.
Your guess is n
Sorry, try again.
Out of guesses. The secret word was whatever
Starting hangman game. Target is _ _ _ _ _ _ _ _ _ _ _ _ _ _ of length 14
You have 6 attempts remaining.
Your guess is i
Good guess: _ _ _ _ _ _ _ i _ _ _ _ _ _
You have 6 attempts remaining.
Your guess i

6

## Bigram Model

Now build a BigramGuesser class exploiting the configuration of the form "\_a" or "a_". For each such configuration in the masked_word, say "\_a", we count for each letter, say "w", the number of occurence of "wa" in the corpus, and measured against the number of occurence of "aa", "ba", etc. For each letter, the bigram frequency is given by the number of bigrams (substring of length two) containing that letter, divided by all possible bigrams. Notice that the BigramGuesser class does not have a guess method: we do not use bigram frequencies to directly predict guesses because by itself it is incapable of producing bigram frequencies when all letters are masked; it only updates the character probabilities. Later we will combine it with unigram model, which is in charge of predicting at least the first letter, in the CombinationGuesser class.

In [160]:
class BigramGuesser(Guesser):
    def __init__(self, by_length = False):
        self.name = "bigram"
        self.map = {}
        self.map_len = {}
        self.char_prob = {}
        self.by_length = by_length

    def adapt(self, corpus):
        self.map = defaultdict(Counter)
        self.map_len = defaultdict(lambda: defaultdict(Counter))
        for word in corpus:
            length = len(word)
            for i in range(length - 1):
                self.map[word[i]][word[i + 1]] += 1
                self.map_len[length][word[i]][word[i + 1]] += 1

    def update_prob(self, masked_word, guessed):
        self.char_prob = {x : 0 for x in string.ascii_lowercase}
        if not self.by_length or len(masked_word) not in self.map_len.keys():
            curr_dict = self.map
        else:
            curr_dict = self.map_len[len(masked_word)]
        total_count = 0
        for i in range(len(masked_word) - 1):
            if masked_word[i] == '_' and masked_word[i + 1] != '_':
                for char in self.char_prob:
                    if char not in guessed:
                        self.char_prob[char] += curr_dict[char][masked_word[i + 1]]
                        total_count += curr_dict[char][masked_word[i + 1]]
            if masked_word[i] != '_' and masked_word[i + 1] == '_':
                for char in self.char_prob:
                    if char not in guessed:
                        self.char_prob[char] += curr_dict[masked_word[i]][char]
                        total_count += curr_dict[masked_word[i]][char]
        if total_count > 0:
            self.char_prob = {x : self.char_prob[x] / total_count for x in self.char_prob}

## Combination Guesser
The combination guesser takes in a sequence of guessers and weights and uses the weighted probabilities to make an informed guess.

In [161]:
class CombinationGuesser(Guesser):
    def __init__(self, guesser_list, weights):
        self.name = ""
        self.char_prob = {}
        self.corpus = []
        self.original_corpus = []
        self.guesser_list = guesser_list
        self.update_weights(weights)

    def upload(self, corpus):
        self.original_corpus = corpus
        self.corpus = corpus
        self.adapt(corpus)

    def adapt(self, corpus):
        self.corpus = corpus
        for guesser in self.guesser_list:
            guesser.adapt(corpus)

    def recalibrate(self, masked_word, guessed):
        incorrect = guessed - set(masked_word)
        self.corpus = [word for word in self.corpus if not set(word).intersection(incorrect)]# and bool(re.fullmatch("".join(masked_word).replace('_', '.'), word))]
        self.adapt(self.corpus)

    def update_by_length(self, lengths):
        for k in range(len(self.guesser_list)):
            self.guesser_list[k].by_length = lengths[k]
        self.update_name()

    def update_name(self):
        self.name = []
        for k in range(len(self.guesser_list)):
            self.name.append(f'{self.weights[k]} * {self.guesser_list[k].name}')
            if self.guesser_list[k].by_length:
                self.name[-1] += "(len)"
        self.name = " + ".join(self.name)

    def update_weights(self, weights):
        self.weights = weights
        self.update_name()

    def update_prob(self, masked_word, guessed):
        self.char_prob = {x : 0 for x in string.ascii_lowercase}
        for guesser in self.guesser_list:
            guesser.update_prob(masked_word, guessed)

    def guess(self, masked_word, guessed):
        self.update_prob(masked_word, guessed)
        for char in self.char_prob:
            for k in range(len(self.guesser_list)):
                self.char_prob[char] += self.guesser_list[k].char_prob[char] * self.weights[k]
        return max(self.char_prob, key=self.char_prob.get)

Let's test our bigram guesser combined with our unigram guesser before. Note that in the first round of guess, bigram probabilities are identically zero so it is all unigram. We see a big improvement from around 18% to 36%. With recalibration, we boost the accuracy to 38%.

In [163]:
bigram_guesser = BigramGuesser()
combination_guesser = CombinationGuesser([unigram_guesser, bigram_guesser], [0.1, 0.9])
combination_guesser.upload(training_set)
test_guesser(combination_guesser, test_set, recalibrate = False, verbose = False)


Testing the 0.1 * unigram + 0.9 * bigram guesser using every word in test set
Average number of guesses used:  8.462
Success rate with 6 tries:  0.358


0.358

In [164]:
hangman('whatever', combination_guesser, 6, recalibrate = True, verbose = True)
hangman('jeepers', combination_guesser, 6, recalibrate = True, verbose = True)
hangman('unquestionably', combination_guesser, 6, recalibrate = True, verbose = True)

Starting hangman game. Target is _ _ _ _ _ _ _ _ of length 8
You have 6 attempts remaining.
Your guess is e
Good guess: _ _ _ _ e _ e _
You have 6 attempts remaining.
Your guess is r
Good guess: _ _ _ _ e _ e r
You have 6 attempts remaining.
Your guess is t
Good guess: _ _ _ t e _ e r
You have 6 attempts remaining.
Your guess is s
Sorry, try again.
You have 5 attempts remaining.
Your guess is n
Sorry, try again.
You have 4 attempts remaining.
Your guess is d
Sorry, try again.
You have 3 attempts remaining.
Your guess is l
Sorry, try again.
You have 2 attempts remaining.
Your guess is a
Good guess: _ _ a t e _ e r
You have 2 attempts remaining.
Your guess is m
Sorry, try again.
You have 1 attempts remaining.
Your guess is h
Good guess: _ h a t e _ e r
You have 1 attempts remaining.
Your guess is c
Sorry, try again.
Out of guesses. The secret word was whatever
Starting hangman game. Target is _ _ _ _ _ _ _ of length 7
You have 6 attempts remaining.
Your guess is e
Good guess: _ e e _ e _

4

Now instead of constructing trigram and 4-gram manually, we write a NgramGuesser class that takes in an input n and gives us an n-gram guesser. The underlying logic is the same as in bigram. Again it does not have a guess method.

## N-gram Guesser

In [129]:
def nested_defaultdict(n):
    return defaultdict(lambda: nested_defaultdict(n - 1)) if n > 0 else Counter()

class NgramGuesser(Guesser):
    def __init__(self, n, by_length = False):
        self.n = n
        self.name = f"{n}-gram"
        self.map = {}
        self.map_len = {}
        self.char_prob = {}
        self.corpus = []
        self.by_length = by_length

    def adapt(self, corpus):
        self.map = nested_defaultdict(self.n - 1)
        self.map_len = nested_defaultdict(self.n)
        for word in corpus:
            length = len(word)
            for i in range(length - self.n + 1):
                ngram_map = self.map
                ngram_map_len = self.map_len[length]
                for k in range(i, i + self.n - 1):
                    ngram_map = ngram_map[word[k]]
                    ngram_map_len = ngram_map_len[word[k]]
                ngram_map[word[i + self.n - 1]] += 1
                ngram_map_len[word[i + self.n - 1]] += 1


    def update_prob(self, masked_word, guessed):
        self.char_prob = {x : 0 for x in string.ascii_lowercase}
        if not self.by_length or len(masked_word) not in self.map_len.keys():
            curr_dict = self.map
        else:
            curr_dict = self.map_len[len(masked_word)]
        total_count = 0
        for i in range(len(masked_word) - self.n + 1):
            if masked_word[i: i + self.n].count('_') == 1:
                idx = masked_word[i: i + self.n].index('_')
                for char in self.char_prob:
                    if char not in guessed:
                        to_add = curr_dict
                        for k in range(i, i + self.n):
                            if k == i + idx:
                                to_add = to_add[char]
                            else:
                                to_add = to_add[masked_word[k]]
                        self.char_prob[char] += to_add
                        total_count += to_add

        if total_count > 0:
            self.char_prob = {x : self.char_prob[x] / total_count for x in self.char_prob}

Let's test on by adding higher-gram model and see how much it can improve.

## 3-gram model

In [186]:
onegram_guesser = NgramGuesser(1)
twogram_guesser = NgramGuesser(2)
threegram_guesser = NgramGuesser(3)
combination_guesser = CombinationGuesser([onegram_guesser, twogram_guesser, threegram_guesser], [0.05, 0.15, 0.8])
combination_guesser.upload(training_set)
test_guesser(combination_guesser, test_set, recalibrate = False, verbose = False)


Testing the 0.05 * 1-gram + 0.15 * 2-gram + 0.8 * 3-gram guesser using every word in test set
Average number of guesses used:  7.635
Success rate with 6 tries:  0.444


0.444

In [170]:
hangman('unquestionably', combination_guesser, 6, recalibrate = False, verbose = True)
hangman('whatever', combination_guesser, 6, recalibrate = False, verbose = True)

Starting hangman game. Target is _ _ _ _ _ _ _ _ _ _ _ _ _ _ of length 14
You have 6 attempts remaining.
Your guess is s
Good guess: _ _ _ _ _ s _ _ _ _ _ _ _ _
You have 6 attempts remaining.
Your guess is e
Good guess: _ _ _ _ e s _ _ _ _ _ _ _ _
You have 6 attempts remaining.
Your guess is t
Good guess: _ _ _ _ e s t _ _ _ _ _ _ _
You have 6 attempts remaining.
Your guess is i
Good guess: _ _ _ _ e s t i _ _ _ _ _ _
You have 6 attempts remaining.
Your guess is l
Good guess: _ _ _ _ e s t i _ _ _ _ l _
You have 6 attempts remaining.
Your guess is n
Good guess: _ n _ _ e s t i _ n _ _ l _
You have 6 attempts remaining.
Your guess is o
Good guess: _ n _ _ e s t i o n _ _ l _
You have 6 attempts remaining.
Your guess is a
Good guess: _ n _ _ e s t i o n a _ l _
You have 6 attempts remaining.
Your guess is b
Good guess: _ n _ _ e s t i o n a b l _
You have 6 attempts remaining.
Your guess is u
Good guess: u n _ u e s t i o n a b l _
You have 6 attempts remaining.
Your guess is q
Good gues

6

With the help of trigram guesser, the success rate improves to around 44%. With recalibration, 46%. One can exploit different hyperparameters here to see what can give the best result: the weights of the individual guesser and whether to take in account the length for each guesser, and whether to recalibrate after each guess of letters. Let's first tweak the weights. From now on, due to the significant cost in time for recalibration, we set recalibration to False.

In [187]:
# searching for the optimal parameter up to 3-gram without considering length
np.random.seed(1)
for i,j in product([0.01, 0.05, 0.1, 0.15, 0.2, 0.25], [0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.4]):
    combination_guesser.update_weights([i, j, 1 - i - j])
    test_guesser(combination_guesser, test_set)


Testing the 0.01 * 1-gram + 0.05 * 2-gram + 0.94 * 3-gram guesser using every word in test set
Average number of guesses used:  7.621
Success rate with 6 tries:  0.444

Testing the 0.01 * 1-gram + 0.1 * 2-gram + 0.89 * 3-gram guesser using every word in test set
Average number of guesses used:  7.656
Success rate with 6 tries:  0.439

Testing the 0.01 * 1-gram + 0.15 * 2-gram + 0.84 * 3-gram guesser using every word in test set
Average number of guesses used:  7.671
Success rate with 6 tries:  0.435

Testing the 0.01 * 1-gram + 0.2 * 2-gram + 0.79 * 3-gram guesser using every word in test set
Average number of guesses used:  7.691
Success rate with 6 tries:  0.436

Testing the 0.01 * 1-gram + 0.25 * 2-gram + 0.74 * 3-gram guesser using every word in test set
Average number of guesses used:  7.662
Success rate with 6 tries:  0.444

Testing the 0.01 * 1-gram + 0.3 * 2-gram + 0.69 * 3-gram guesser using every word in test set
Average number of guesses used:  7.665
Success rate with 6 tri

We see that there isn't much fundamental differences as all results lie in [43%, 46%]. Small 1-gram and 2-gram weights seem to generate the best result closer to the upper bound.

In [147]:
combination_guesser.update_weights([0.01, 0.1, 0.89])
for i, j, k in product([False, True], [False, True], [False, True]):
    combination_guesser.update_by_length([i, j, k])
    test_guesser(combination_guesser, test_set)


Testing the 0.01 * 1-gram + 0.1 * 2-gram + 0.89 * 3-gram guesser using every word in test set
Average number of guesses used:  7.656
Success rate with 6 tries:  0.439

Testing the 0.01 * 1-gram + 0.1 * 2-gram + 0.89 * 3-gram(len) guesser using every word in test set
Average number of guesses used:  7.706
Success rate with 6 tries:  0.449

Testing the 0.01 * 1-gram + 0.1 * 2-gram(len) + 0.89 * 3-gram guesser using every word in test set
Average number of guesses used:  7.663
Success rate with 6 tries:  0.443

Testing the 0.01 * 1-gram + 0.1 * 2-gram(len) + 0.89 * 3-gram(len) guesser using every word in test set
Average number of guesses used:  7.76
Success rate with 6 tries:  0.435

Testing the 0.01 * 1-gram(len) + 0.1 * 2-gram + 0.89 * 3-gram guesser using every word in test set
Average number of guesses used:  7.636
Success rate with 6 tries:  0.438

Testing the 0.01 * 1-gram(len) + 0.1 * 2-gram + 0.89 * 3-gram(len) guesser using every word in test set
Average number of guesses used:

We are able to boost the success rate close to 47% and we see that conditioning on length seems only beneficial for 1-gram but not so much for higher-gram.One reasonable explanation is that after conditioning on length there aren't much higher grams left to train on. Let's proceed to higher-gram and see how far we can go. From now on we only turn on length conditioning for 1-gram.

## 4-gram model

In [178]:
fourgram_guesser = NgramGuesser(4)
combination_guesser = CombinationGuesser([onegram_guesser, twogram_guesser, threegram_guesser, fourgram_guesser], [0.01, 0.02, 0.07, 0.9])
combination_guesser.upload(training_set)
combination_guesser.update_by_length([True, False, False, False])
test_guesser(combination_guesser, test_set, recalibrate = False, verbose = False)


Testing the 0.01 * 1-gram(len) + 0.02 * 2-gram + 0.07 * 3-gram + 0.9 * 4-gram guesser using every word in test set
Average number of guesses used:  6.654
Success rate with 6 tries:  0.54


0.54

In [169]:
hangman('whatever', combination_guesser, 6, recalibrate = False, verbose = True)
hangman('unquestionably', combination_guesser, 6, recalibrate = False, verbose = True)

Starting hangman game. Target is _ _ _ _ _ _ _ _ of length 8
You have 6 attempts remaining.
Your guess is s
Sorry, try again.
You have 5 attempts remaining.
Your guess is e
Good guess: _ _ _ _ e _ e _
You have 5 attempts remaining.
Your guess is l
Sorry, try again.
You have 4 attempts remaining.
Your guess is n
Sorry, try again.
You have 3 attempts remaining.
Your guess is m
Sorry, try again.
You have 2 attempts remaining.
Your guess is t
Good guess: _ _ _ t e _ e _
You have 2 attempts remaining.
Your guess is p
Sorry, try again.
You have 1 attempts remaining.
Your guess is a
Good guess: _ _ a t e _ e _
You have 1 attempts remaining.
Your guess is h
Good guess: _ h a t e _ e _
You have 1 attempts remaining.
Your guess is w
Good guess: w h a t e _ e _
You have 1 attempts remaining.
Your guess is x
Sorry, try again.
Out of guesses. The secret word was whatever
Starting hangman game. Target is _ _ _ _ _ _ _ _ _ _ _ _ _ _ of length 14
You have 6 attempts remaining.
Your guess is s
Good gue

0

In [ ]:
for i, j, k in product([0.01, 0.02, 0.05, 0.1, 0.2], [0.02, 0.05, 0.1, 0.15, 0.2], [0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.4]):
    combination_guesser.update_weights([i, j, k, 1 - i - j - k])
    test_guesser(combination_guesser, test_set)


Testing the 0.01 * 1-gram(len) + 0.02 * 2-gram + 0.05 * 3-gram + 0.9199999999999999 * 4-gram guesser using every word in test set
Average number of guesses used:  6.345
Success rate with 6 tries:  0.571

Testing the 0.01 * 1-gram(len) + 0.02 * 2-gram + 0.1 * 3-gram + 0.87 * 4-gram guesser using every word in test set
Average number of guesses used:  6.348
Success rate with 6 tries:  0.571

Testing the 0.01 * 1-gram(len) + 0.02 * 2-gram + 0.15 * 3-gram + 0.82 * 4-gram guesser using every word in test set
Average number of guesses used:  6.347
Success rate with 6 tries:  0.571

Testing the 0.01 * 1-gram(len) + 0.02 * 2-gram + 0.2 * 3-gram + 0.77 * 4-gram guesser using every word in test set
Average number of guesses used:  6.369
Success rate with 6 tries:  0.569

Testing the 0.01 * 1-gram(len) + 0.02 * 2-gram + 0.25 * 3-gram + 0.72 * 4-gram guesser using every word in test set
Average number of guesses used:  6.365
Success rate with 6 tries:  0.573

Testing the 0.01 * 1-gram(len) + 0.02

Now we boost our success rate up to 58.7% with 4-gram! It is achieved at weights [0.1, 0.2, 0.15, 0.55]. The worst is around 52% when the weight for the 4-gram is only at 0.2. This suggests turning on 4-gram really helps our model!

## 5-gram model

In [179]:
fivegram_guesser = NgramGuesser(5)
combination_guesser = CombinationGuesser([onegram_guesser, twogram_guesser, threegram_guesser, fourgram_guesser, fivegram_guesser], [0.01, 0.02, 0.02, 0.05, 0.9])
combination_guesser.upload(training_set)
combination_guesser.update_by_length([True, False, False, False, False])
test_guesser(combination_guesser, test_set, recalibrate = False, verbose = False)


Testing the 0.01 * 1-gram(len) + 0.02 * 2-gram + 0.02 * 3-gram + 0.05 * 4-gram + 0.9 * 5-gram guesser using every word in test set
Average number of guesses used:  6.077
Success rate with 6 tries:  0.626


0.626

In [173]:
hangman('whatever', combination_guesser, 6, recalibrate = False, verbose = True)
hangman('unquestionably', combination_guesser, 6, recalibrate = False, verbose = True)

Starting hangman game. Target is _ _ _ _ _ _ _ _ of length 8
You have 6 attempts remaining.
Your guess is e
Good guess: _ _ _ _ e _ e _
You have 6 attempts remaining.
Your guess is n
Sorry, try again.
You have 5 attempts remaining.
Your guess is l
Sorry, try again.
You have 4 attempts remaining.
Your guess is s
Sorry, try again.
You have 3 attempts remaining.
Your guess is m
Sorry, try again.
You have 2 attempts remaining.
Your guess is d
Sorry, try again.
You have 1 attempts remaining.
Your guess is t
Good guess: _ _ _ t e _ e _
You have 1 attempts remaining.
Your guess is v
Good guess: _ _ _ t e v e _
You have 1 attempts remaining.
Your guess is h
Good guess: _ h _ t e v e _
You have 1 attempts remaining.
Your guess is i
Sorry, try again.
Out of guesses. The secret word was whatever
Starting hangman game. Target is _ _ _ _ _ _ _ _ _ _ _ _ _ _ of length 14
You have 6 attempts remaining.
Your guess is i
Good guess: _ _ _ _ _ _ _ i _ _ _ _ _ _
You have 6 attempts remaining.
Your guess i

0

In [ ]:
best_accuracy, worst_accuracy = 0, 1
best_weights, worst_weights = [], []
for i, j, k, l in product([0.01, 0.05, 0.1, 0.2], [0.02, 0.05, 0.1, 0.2], [0.1, 0.2, 0.3, 0.4, 0.5], [0.1, 0.2, 0.3, 0.4, 0.5]):
    if i + j + k + l > 1:
        continue
    combination_guesser.update_weights([i, j, k, l, 1 - i - j - k - l])
    result = test_guesser(combination_guesser, test_set)
    if result > best_accuracy:
        best_accuracy = result
        best_weights = [i, j, k, l, 1 - i - j - k - l]
    if result < worst_accuracy:
        worst_accuracy = result
        worst_weights = [i, j, k, l, 1 - i - j - k - l]
print(f"Best success rate is {best_accuracy} and is achieved with weights {best_weights}")
print(f"Worst success rate is {worst_accuracy} and is achieved with weights {worst_weights}")


Testing the 0.01 * 1-gram(len) + 0.02 * 2-gram + 0.1 * 3-gram + 0.1 * 4-gram + 0.77 * 5-gram guesser using every word in test set
Average number of guesses used:  5.948
Success rate with 6 tries:  0.618

Testing the 0.01 * 1-gram(len) + 0.02 * 2-gram + 0.1 * 3-gram + 0.2 * 4-gram + 0.6699999999999999 * 5-gram guesser using every word in test set
Average number of guesses used:  5.928
Success rate with 6 tries:  0.623

Testing the 0.01 * 1-gram(len) + 0.02 * 2-gram + 0.1 * 3-gram + 0.3 * 4-gram + 0.5700000000000001 * 5-gram guesser using every word in test set
Average number of guesses used:  5.932
Success rate with 6 tries:  0.62

Testing the 0.01 * 1-gram(len) + 0.02 * 2-gram + 0.1 * 3-gram + 0.4 * 4-gram + 0.47 * 5-gram guesser using every word in test set
Average number of guesses used:  5.934
Success rate with 6 tries:  0.62

Testing the 0.01 * 1-gram(len) + 0.02 * 2-gram + 0.1 * 3-gram + 0.5 * 4-gram + 0.37 * 5-gram guesser using every word in test set
Average number of guesses u

A success rate of 63.4% is achieved at weights [0.05, 0.1, 0.1, 0.4, 0.35] or [0.1, 0.2, 0.1, 0.3, 0.3] or [0.1, 0.2, 0.1, 0.4, 0.2], all with a good mix of 4-gram and 5-gram. We start to see diminishing returns, but let's see how far we can push it.

## 6-gram model

In [180]:
sixgram_guesser = NgramGuesser(6)
combination_guesser = CombinationGuesser([onegram_guesser, twogram_guesser, threegram_guesser, fourgram_guesser, fivegram_guesser, sixgram_guesser], [0.05, 0.05, 0.1, 0.1, 0.2, 0.5])
combination_guesser.upload(training_set)
combination_guesser.update_by_length([True, False, False, False, False, False])
test_guesser(combination_guesser, test_set, recalibrate = False, verbose = False)


Testing the 0.05 * 1-gram(len) + 0.05 * 2-gram + 0.1 * 3-gram + 0.1 * 4-gram + 0.2 * 5-gram + 0.5 * 6-gram guesser using every word in test set
Average number of guesses used:  5.931
Success rate with 6 tries:  0.623


0.623

In [ ]:
best_accuracy, worst_accuracy = 0, 1
best_weights, worst_weights = [], []
for i, j, k, l, u in product([0.01, 0.05, 0.1], [0.05, 0.1, 0.2, 0.3], [0.1, 0.2, 0.3], [0.1, 0.2, 0.3, 0.4], [0.1, 0.2, 0.3, 0.4, 0.5]):
    if i + j + k + l + u > 1:
        continue
    combination_guesser.update_weights([i, j, k, l, u, 1 - i - j - k - l- u])
    result = test_guesser(combination_guesser, test_set)
    if result > best_accuracy:
        best_accuracy = result
        best_weights = [i, j, k, l, u, 1 - i - j - k - l - u]
    if result < worst_accuracy:
        worst_accuracy = result
        worst_weights = [i, j, k, l, u, 1 - i - j - k - l - u]
print(f"Best success rate is {best_accuracy} and is achieved with weights {best_weights}")
print(f"Worst success rate is {worst_accuracy} and is achieved with weights {worst_weights}")



Testing the 0.01 * 1-gram(len) + 0.05 * 2-gram + 0.1 * 3-gram + 0.1 * 4-gram + 0.1 * 5-gram + 0.64 * 6-gram guesser using every word in test set
Average number of guesses used:  5.719
Success rate with 6 tries:  0.631

Testing the 0.01 * 1-gram(len) + 0.05 * 2-gram + 0.1 * 3-gram + 0.1 * 4-gram + 0.2 * 5-gram + 0.54 * 6-gram guesser using every word in test set
Average number of guesses used:  5.742
Success rate with 6 tries:  0.635

Testing the 0.01 * 1-gram(len) + 0.05 * 2-gram + 0.1 * 3-gram + 0.1 * 4-gram + 0.3 * 5-gram + 0.44 * 6-gram guesser using every word in test set
Average number of guesses used:  5.762
Success rate with 6 tries:  0.635

Testing the 0.01 * 1-gram(len) + 0.05 * 2-gram + 0.1 * 3-gram + 0.1 * 4-gram + 0.4 * 5-gram + 0.33999999999999997 * 6-gram guesser using every word in test set
Average number of guesses used:  5.775
Success rate with 6 tries:  0.633

Testing the 0.01 * 1-gram(len) + 0.05 * 2-gram + 0.1 * 3-gram + 0.1 * 4-gram + 0.5 * 5-gram + 0.24 * 6-gram 

Best success rate is 0.66 and is achieved with weights [0.1, 0.2, 0.1, 0.2, 0.1, 0.3]. This is only a 3% increase compared to 5-gram.

## 7-gram model

In [181]:
sevengram_guesser = NgramGuesser(7)
combination_guesser = CombinationGuesser([onegram_guesser, twogram_guesser, threegram_guesser, fourgram_guesser, fivegram_guesser, sixgram_guesser, sevengram_guesser], [0.05, 0.05, 0.1, 0.1, 0.2, 0.2, 0.3])
combination_guesser.upload(training_set)
combination_guesser.update_by_length([True, False, False, False, False, False, False])
test_guesser(combination_guesser, test_set, recalibrate = False, verbose = False)


Testing the 0.05 * 1-gram(len) + 0.05 * 2-gram + 0.1 * 3-gram + 0.1 * 4-gram + 0.2 * 5-gram + 0.2 * 6-gram + 0.3 * 7-gram guesser using every word in test set
Average number of guesses used:  5.914
Success rate with 6 tries:  0.624


0.624

In [ ]:
best_accuracy, worst_accuracy = 0, 1
best_weights, worst_weights = [], []
for i, j, k, l, u, v in product([0.05, 0.1, 0.2], [0.05, 0.1, 0.2, 0.3], [0.1, 0.2, 0.3], [0.1, 0.2, 0.3], [0.1, 0.2, 0.3, 0.4], [0.1, 0.2, 0.3, 0.4]):
    if i + j + k + l + u + v > 1:
        continue
    combination_guesser.update_weights([i, j, k, l, u, v, 1 - i - j - k - l - u - v])
    result = test_guesser(combination_guesser, test_set)
    if result > best_accuracy:
        best_accuracy = result
        best_weights = [i, j, k, l, u, v, 1 - i - j - k - l - u - v]
    if result < worst_accuracy:
        worst_accuracy = result
        worst_weights = [i, j, k, l, u, v, 1 - i - j - k - l - u - v]
print(f"Best success rate is {best_accuracy} and is achieved with weights {best_weights}")
print(f"Worst success rate is {worst_accuracy} and is achieved with weights {worst_weights}")


Testing the 0.05 * 1-gram(len) + 0.05 * 2-gram + 0.1 * 3-gram + 0.1 * 4-gram + 0.1 * 5-gram + 0.1 * 6-gram + 0.5 * 7-gram guesser using every word in test set
Average number of guesses used:  5.703
Success rate with 6 tries:  0.635

Testing the 0.05 * 1-gram(len) + 0.05 * 2-gram + 0.1 * 3-gram + 0.1 * 4-gram + 0.1 * 5-gram + 0.2 * 6-gram + 0.39999999999999997 * 7-gram guesser using every word in test set
Average number of guesses used:  5.701
Success rate with 6 tries:  0.636

Testing the 0.05 * 1-gram(len) + 0.05 * 2-gram + 0.1 * 3-gram + 0.1 * 4-gram + 0.1 * 5-gram + 0.3 * 6-gram + 0.3 * 7-gram guesser using every word in test set
Average number of guesses used:  5.697
Success rate with 6 tries:  0.637

Testing the 0.05 * 1-gram(len) + 0.05 * 2-gram + 0.1 * 3-gram + 0.1 * 4-gram + 0.1 * 5-gram + 0.4 * 6-gram + 0.19999999999999996 * 7-gram guesser using every word in test set
Average number of guesses used:  5.7
Success rate with 6 tries:  0.638

Testing the 0.05 * 1-gram(len) + 0.05

As you can see, adding 7-gram stops improving for the model. The best success rate stayed at 66% with weights [0.1, 0.2, 0.1, 0.2, 0.1, 0.3, 0] and [0.1, 0.2, 0.1, 0.2, 0.1, 0.2, 0.1]. So, let's stop at 6-gram! Below we record our best model weights.

In [182]:
combination_guesser = CombinationGuesser([onegram_guesser, twogram_guesser, threegram_guesser, fourgram_guesser, fivegram_guesser, sixgram_guesser], [0.1, 0.2, 0.1, 0.2, 0.1, 0.3])
combination_guesser.upload(training_set)
combination_guesser.update_by_length([True, False, False, False, False, False])
test_guesser(combination_guesser, test_set)


Testing the 0.1 * 1-gram(len) + 0.2 * 2-gram + 0.1 * 3-gram + 0.2 * 4-gram + 0.1 * 5-gram + 0.3 * 6-gram guesser using every word in test set
Average number of guesses used:  5.945
Success rate with 6 tries:  0.64


0.64

In [188]:
hangman('whatever', combination_guesser, 6, recalibrate = False, verbose = True)
hangman('unquestionably', combination_guesser, 6, recalibrate = False, verbose = True)

Starting hangman game. Target is _ _ _ _ _ _ _ _ of length 8
You have 6 attempts remaining.
Your guess is e
Good guess: _ _ _ _ e _ e _
You have 6 attempts remaining.
Your guess is r
Good guess: _ _ _ _ e _ e r
You have 6 attempts remaining.
Your guess is t
Good guess: _ _ _ t e _ e r
You have 6 attempts remaining.
Your guess is n
Sorry, try again.
You have 5 attempts remaining.
Your guess is s
Sorry, try again.
You have 4 attempts remaining.
Your guess is d
Sorry, try again.
You have 3 attempts remaining.
Your guess is a
Good guess: _ _ a t e _ e r
You have 3 attempts remaining.
Your guess is l
Sorry, try again.
You have 2 attempts remaining.
Your guess is m
Sorry, try again.
You have 1 attempts remaining.
Your guess is c
Sorry, try again.
Out of guesses. The secret word was whatever
Starting hangman game. Target is _ _ _ _ _ _ _ _ _ _ _ _ _ _ of length 14
You have 6 attempts remaining.
Your guess is e
Good guess: _ _ _ _ e _ _ _ _ _ _ _ _ _
You have 6 attempts remaining.
Your guess i

4